<img src="DXP.PNG" style="width:350px;height:275px;">
<h2 style="color:red">长 江 黄 河 永 不 倒 流 、改 开 之 路 绝 不 向 西 ！ 敬 爱 的 鄧 小 平 同 志 永 远 活 在 我 们 心 中</h2>
<h5 style="color:red"> - 二零二三年二月十九日</h5>

# 广安级（第三代）机器学习调参算法

In [2]:
# 17/2/2023

In [3]:
import pandas as pd
import numpy as np
import copy
import statsmodels.api as sm
import pickle
import time

from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [23]:
class GuangAn:



    def __init__(self):
        """ Initialise class """
        self._initialise_objects()

        print('GuangAn Initialised')



    def _initialise_objects(self):
        """ Helper to initialise objects """

        self.train_x = None
        self.train_y = None
        self.val_x = None
        self.val_y = None
        self.test_x = None
        self.test_y = None
        # self.tuning_result = None
        self.model = None
        # self.parameter_choices = None
        # self.hyperparameters = None
        # self.checked = None
        # self.result = None
        # self.tuning_result_saving_address = None
        # self.object_saving_address = None
        self._up_to = 0
        # self._seed = 19421221
        # self.best_score = -np.inf
        # self.best_combo = None
        # self.best_clf = None
        self.clf_type = None


        self.regression_extra_output_columns = ['Train r2', 'Val r2', 'Test r2', 
            'Train RMSE', 'Val RMSE', 'Test RMSE', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Time']
        self.classification_extra_output_columns = ['Train accu', 'Val accu', 'Test accu', 
            'Train balanced_accu', 'Val balanced_accu', 'Test balanced_accu', 'Train f1', 'Val f1', 'Test f1', 
            'Train precision', 'Val precision', 'Test precision', 'Train recall', 'Val recall', 'Test recall', 'Time']

        

    def read_in_data(self, train_x, train_y, val_x, val_y, test_x, test_y):
        """ Reads in train validate test data for tuning """

        self.train_x = train_x
        print("Read in Train X data")

        self.train_y = train_y
        print("Read in Train x data")

        self.val_x = val_x
        print("Read in Val X data")

        self.val_y = val_y
        print("Read in Val y data")

        self.test_x = test_x
        print("Read in Test X data")

        self.test_y = test_y
        print("Read in Test y data")



    def read_in_model(self, model, type):
        """ Reads in underlying model object for tuning, and also read in what type of model it is """

        assert type == 'Classification' or type == 'Regression' # check

        # record
        self.model = model
        self.clf_type = type 

        print(f'Successfully read in model {self.model}, which is a {self.clf_type} model')



    def set_hyperparameters(self, parameter_ranges_orig):
        """ Input hyperparameter choices """

        self.parameter_ranges = parameter_ranges_orig
        self._sort_hyperparameter_ranges()
        

        self.hyperparameters = list(self.parameter_ranges.keys())

        self.categorical = {hp:False for hp in self.hyperparameters}
        self.transform = {hp:False for hp in self.hyperparameters}

        self._get_checked_dict()
        self._setup_tuning_result_df()

        self.original_bounds = [(self.parameter_ranges[key], key) for key in self.parameter_ranges]

        print("Successfully recorded hyperparameter choices")



    def _sort_hyperparameter_ranges(self):
        """ Helper to ensure all hyperparameter choice values are in order from lowest to highest """

        for key in self.parameter_ranges:
            tmp = copy.deepcopy(list(self.parameter_ranges[key]))
            tmp.sort()
            if type(self.parameter_ranges[key]) is set:
                self.parameter_ranges[key] = set(tmp)
            else:
                self.parameter_ranges[key] = tuple(tmp)


    
    def _setup_tuning_result_df(self):
        """ Helper to set up tuning result dataframe """

        tune_result_columns = copy.deepcopy(self.hyperparameters)

        # Different set of metric columns for different types of models
        if self.clf_type == 'Classification':
            tune_result_columns.extend(self.classification_extra_output_columns)
        elif self.clf_type == 'Regression':
            tune_result_columns.extend(self.regression_extra_output_columns)

        self.tuning_result = pd.DataFrame({col:list() for col in tune_result_columns})

        
    
    def _get_checked_dict(self):
        """ Helper to set up checked list """

        self.checked_dict = dict()



    def set_non_tuneable_hyperparameters(self, non_tuneable_hyperparameter_choice):
        """ Input Non tuneable hyperparameter choice """

        if type(non_tuneable_hyperparameter_choice) is not dict:
            print('non_tuneable_hyeprparameters_choice must be dict, please try again')
            return
        
        for nthp in non_tuneable_hyperparameter_choice:
            if type(non_tuneable_hyperparameter_choice[nthp]) in (set, list, tuple, dict):
                print('non_tuneable_hyperparameters_choice must not be of array-like type')
                return

        self.non_tuneable_parameter_choices = non_tuneable_hyperparameter_choice

        print("Successfully recorded non_tuneable_hyperparameter choices")



    def read_in_transform(self, transform_update):
        """ Function to read in transformation settings """

        if type(transform_update) is not dict:
            print('transform_update should be a dict, please re-enter')
            return

        for key in transform_update:
            self.transform[key] = transform_update[key]

        print('Updated transform dictionary:', self.transform)

    

    def read_in_categorical(self, categorical_update):
        """ Function to read in categorical settings """

        if type(categorical_update) is not list:
            print('categorical_update should be a list, please re-enter')
            return

        for key in categorical_update:
            self.categorical[key] = True
            
            self.parameter_ranges[key] = {'values': tuple(self.parameter_ranges[key])}
        
        self.original_bounds = [(self.parameter_ranges[key], key) for key in self.parameter_ranges]

        print('Updated categorical dictionary:', self.categorical)
        print('Updated original bounds dict:', self.original_bounds)

    

    def set_features(self, ningxiang_output):
        """ Input features """

        if type(ningxiang_output) is not dict:
            print("Please ensure NingXiang output is a dict")
            return
        
        if not self.hyperparameters:
            print("Missing hyperparameter choices, please run .set_hyperparameters() first")
            return
        
        # sort ningxiang just for safety, and store up
        ningxiang_output_sorted = self._sort_features(ningxiang_output)
        self.feature_n_ningxiang_score_dict = ningxiang_output_sorted

        # activate this switch
        self._tune_features = True

        # update previous internal structures based on first set of hyperparameter choices
        ##here used numbers instead of tuples as the values in parameter_choices; thus need another mapping to get map back to the features
        self.parameter_choices['features'] = set([i for i in range(len(ningxiang_output_sorted))])
        self.feature_combo_n_index_map = {i: list(ningxiang_output_sorted.keys())[i] for i in range(len(ningxiang_output_sorted))}

        self.hyperparameters = list(self.parameter_ranges.keys())
        
        self.categorical['features'] = False
        self.transform['features'] = False

        self._get_checked_dict()
        self._setup_tuning_result_df()

        self.original_bounds = [(self.parameter_ranges[key], key) for key in self.parameter_ranges]

        print("Successfully recorded tuneable feature combination choices and updated relevant internal structures")


    
    def _sort_features(self, ningxiang_output):
        """ Helper for sorting features based on NingXiang values (input dict output dict) """

        ningxiang_output_list = [(key, ningxiang_output[key]) for key in ningxiang_output]

        ningxiang_output_list.sort(key = lambda x:x[1])

        ningxiang_output_sorted = {x[0]:x[1] for x in ningxiang_output_list}

        return ningxiang_output_sorted



    def get_coords_from_bounds(bounds):
        """ Function to get initial coordinates to tune """

        # Setup the initial list that is used in classic JiaXing combo getting algorithm
        if type(bounds[0][0]) is tuple:
            boundary_coordinates = [[bounds[0][0][i]] for i in range(2)]
        elif type(bounds[0][0]) is set: 
            if len(bounds[0][0]) == 1:
                boundary_coordinates = [[list(bounds[0][0])[0]]]
            else:
                boundary_coordinates = [[list(bounds[0][0])[0]], [list(bounds[0][0])[-1]]]
        elif type(bounds[0][0]) is dict:
            boundary_coordinates = [[bounds[0][0]['values'][i]] for i in range(len(bounds[0][0]['values']))]

        # Second part of classic JiaXing combo getting algorithm
        for i in range(1, len(bounds)):
            old_boundary_coordinates = copy.deepcopy(boundary_coordinates)
            boundary_coordinates = list()

            values = bounds[i]

            for init_coord in old_boundary_coordinates:
                if type(values[0]) is tuple: # tuple: continuous values
                    for value in values[0]:
                        tmp = copy.deepcopy(init_coord)
                        tmp.append(value)
                        boundary_coordinates.append(tmp)

                elif type(values[0]) is set: # set: semi-continuous values (ordinal or floats but not continuous) 
                    if len(values[0]) == 1:
                        tmp = copy.deepcopy(init_coord)
                        tmp.append(list(values[0])[0])
                        boundary_coordinates.append(tmp)
                    else:
                        for value in [list(values[0])[0], list(values[0])[-1]]:
                            tmp = copy.deepcopy(init_coord)
                            tmp.append(value)
                            boundary_coordinates.append(tmp)
                
                elif type(values[0]) is dict: # dict: discrete values
                    for value in values[0]['values']:
                        tmp = copy.deepcopy(init_coord)
                        tmp.append(value)
                        boundary_coordinates.append(tmp)

        return boundary_coordinates



    def get_centre_components(self, bounds, categorical):
        """ Helper that gets the centres of a bound as lists (considering for categorical) """

        # Classic JiaXing getting combo algorithm
        centre_components = list()
        tmp_cat = copy.deepcopy(categorical)
        for i in range(len(bounds)):

            if type(bounds[i][0]) is tuple: # tuple: continuous values
                # take the mean value
                centre_components.append(sum(bounds[i][0])/2)

            elif type(bounds[i][0]) is set: # set: semi-continuous values (ordinal or floats but not continuous) 
                n_semicont_values = len(bounds[i][0])
                if n_semicont_values <= 2:

                    # just input the tuple(set) as the centre (which will be recognised as two discrete)
                    centre_components.append(tuple(bounds[i][0]))

                    # set categorical of this variable to true because no more semi-continuous values in between the current bounds
                    tmp_cat[self.hyperparameters[i]] = True 

                else:
                    # take the middle
                    centre_components.append(list(bounds[i][0])[n_semicont_values//2])

            elif type(bounds[i][0]) is dict: # dict: discrete values
                # input the value (a set) which will be recognised as discrete
                centre_components.append(bounds[i][0]['values'])
        
        # returns 1. components that can be unpacked into multiple centres; 2. new categorical labels
        return centre_components, tmp_cat



    def unpack_centre(self, centre_components):
        """ Helper to unpack centre components into centre """

        # Classic JiaXing algorithm for getting all combinations
        centres = [[]]
        for i in range(len(centre_components)):
            old_centres = copy.deepcopy(centres)
            centres = list()
            if type(centre_components[i]) is tuple:
                for obj in centre_components[i]:
                    for cent in old_centres:
                        tmp_cent = copy.deepcopy(cent)
                        tmp_cent.append(obj)
                        centres.append(tmp_cent)
            else:
                for cent in old_centres:
                    tmp_cent = copy.deepcopy(cent)
                    tmp_cent.append(centre_components[i])
                    centres.append(tmp_cent)

        return [tuple(centre) for centre in centres]


    
    def get_categorical(self, new_cat, boundaries):
        """ Helper to get all combos of categorical feature's values (for use in OLS) """

        # Classic JiaXing algorithm for getting all combinations
        out = [[]]
        for hyperparameter in new_cat:
            if new_cat[hyperparameter] is True:
                old_out = copy.deepcopy(out)
                out = list()

                val_list = list(boundaries[hyperparameter])
                val_list_unique = list(set(val_list))
                val_list_unique.sort()

                for val in val_list_unique:
                    for lst in old_out:
                        tmp = copy.deepcopy(lst)
                        tmp.append(val)
                        out.append(tmp)
        
        return out



    def get_new_bounds(self, bounds, centre, categorical):
        """ Function to get new bounds """

        # get the range components that make the 2^d bounds
        range_components = self.get_range_components(bounds, centre, categorical)

        # make the new bounds from components
        new_bounds = self.make_bounds(range_components)

        return new_bounds



    def get_range_components(self, bounds, centre, categorical):
        """ Helper that gets the range components """

        range_components = list()
        for i in range(len(bounds)):
            
            if type(bounds[i][0]) is tuple:
                lower_range = (bounds[i][0][0], centre[i]) 
                upper_range = (centre[i], bounds[i][0][1]) 

                ranges = (lower_range, upper_range)

                range_components.append((ranges, bounds[i][1]))
            
            elif type(bounds[i][0]) is set:
                
                if categorical[self.hyperparameters[i]] is False:
                    lower_range_min_max = (list(bounds[i][0])[0], centre[i]) 
                    upper_range_min_max = (centre[i], list(bounds[i][0])[-1]) 
                    lower_range = list()
                    for orig_val in list(self.parameter_ranges[self.hyperparameters[i]]):
                        if orig_val <= lower_range_min_max[1] and lower_range_min_max[0] <= orig_val:
                            lower_range.append(orig_val)
                    lower_range.sort()
                    lower_range = set(lower_range)
                    
                    upper_range = list()
                    for orig_val in list(self.parameter_ranges[self.hyperparameters[i]]):
                        if orig_val <= upper_range_min_max[1] and upper_range_min_max[0] <= orig_val:
                            upper_range.append(orig_val)
                    upper_range.sort()
                    upper_range = set(upper_range)

                    ranges = (lower_range, upper_range)

                    range_components.append((ranges, bounds[i][1]))
                
                else:
                    ranges = bounds[i][0]
                    range_components.append((ranges, bounds[i][1]))


            elif type(bounds[i][0]) is dict:
                
                ranges = set(bounds[i][0]['values'])

                range_components.append((ranges, bounds[i][1]))
        
        return range_components



    def make_bounds(self, range_components):
        """ Helper that makes the bounds using range components """

        # Algorithm to create all bounds
        if type(range_components[0][0]) is tuple:
            bounds = [[(range_components[0][0][i], range_components[0][1])] for i in range(2)] # hardcode cos bounds can only have 2 values
        elif type(range_components[0][0]) is set:
            tmp_tup = tuple(range_components[0][0])
            bounds = [[({tmp_tup[i]}, range_components[0][1])] for i in range(len(tmp_tup))]

        for i in range(1, len(range_components)):
            old_bounds = copy.deepcopy(bounds)
            bounds = list()

            values = range_components[i]

            for bound in old_bounds:
                
                if type(values[0]) == tuple:
                    for value in values[0]:
                        tmp = copy.deepcopy(bound)
                        tmp.append((value, values[1]))

                        bounds.append(tmp)
                
                elif type(values[0]) == set:
                    for value in list(values[0]):
                        tmp = copy.deepcopy(bound)
                        tmp.append(({value}, values[1]))

                        bounds.append(tmp)
        
        return bounds



    def rebuild_bounds_to_original_format(self, tmp_boundary, new_cat):
        """ Helper to rebuild current format of bounds (as a df) into original bound format"""

        tmp_boundary = tmp_boundary.drop(['score'], axis = 1)
        
        bounds_original_format = list()
        for col in tmp_boundary.columns:

        # if already categorical: just keep it as categorical
            if new_cat[col] == True: 
                bounds_original_format.append(({'values': tuple(set(tmp_boundary[col]))}, col))
        
            else:
                col_vals = list(set(tmp_boundary[col]))
            
                if type(self.parameter_ranges[col]) is set:
                    tmp = list()
                    curr_val_max = max(col_vals)
                    curr_val_min = min(col_vals)
                    for orig_val in list(self.parameter_ranges[col]):
                        if orig_val <= curr_val_max and curr_val_min <= orig_val:
                            tmp.append(orig_val)
                    tmp.sort()
                    tmp = set(tmp)
                    bounds_original_format.append((tmp, col))

                else: # continuous values
                    bounds_original_format.append(((min(col_vals), max(col_vals)), col))

        return bounds_original_format



    def _get_list_from_df(self, df):
        """ Helper to get df rows into list form """
        
        out = list()
        for row in df.iterrows():
            out.append(list(row[1].values))
        
        return out



    def _get_protective_bounds(self, bounds):
        """ Helper to get protective bounds  - for boundaries"""

        protective_bounds = list()
        for bound in bounds:
            if type(bound[0]) is dict: # categorical values become a set
                protective_bounds.append(set(bound[0]['values']))
            elif type(bound[0]) is set: # semi_categorical values become tuple
                protective_bounds.append((list(bound[0])[0], list(bound[0])[-1]))
            elif type(bound[0]) is tuple: # continuous values stay as tuple
                protective_bounds.append(bound[0])
        
        return protective_bounds



    def _get_protective_bounds2(self, tmp_boundary, new_cat):
        """ Helper to get protective bounds - for centres """

        protective_bounds = list()
        for col in tmp_boundary.columns:
            if col == 'score':
                 continue
            
            col_values = list(tmp_boundary[col])

            if new_cat[col]: # categorical values - only one left - set
                protective_bounds.append({col_values[0],})
            
            else:  # continuous values - tuple
                protective_bounds.append([min(col_values), max(col_values)])
                 
        
        return protective_bounds
    


    def _in_protective_bounds(self, centre):
        """ Determine whether centre is in protective_bounds """

        for i in range(len(centre)):
            if type(self._protective_bounds[i]) is set: # categorical
                if centre[i] not in self._protective_bounds[i]: # not matching any of the categorical values
                    return False 
            else: # continuous values
                if centre[i] > self._protective_bounds[i][1] or centre[i] < self._protective_bounds[i][0]: # outside of boundary tuple
                    return False
        
        return True
    


    def _protective_bounds_to_original_bounds(self):
        """ Helper to turn protective bounds back to original bounds """

        protective_to_original_bounds = list()
        for i in range(len(self._protective_bounds)):
            if type(self._protective_bounds[i]) is set: # discrete
                protective_to_original_bounds.append(({'values': tuple(self._protective_bounds[i])}, self.original_bounds[i][1]))
            else:
                if type(self.original_bounds[i][0]) is set:
                    
                    tmp = list()
                    for val in list(self.original_bounds[i][0]):
                        if val <= self._protective_bounds[i][1] and val >= self._protective_bounds[i][0]:
                            tmp.append(val)

                    protective_to_original_bounds.append((set(val), self.original_bounds[i][1]))
                else:
                    protective_to_original_bounds.append((self._protective_bounds[i], self.original_bounds[i][1]))
        
        return protective_to_original_bounds



    def tune(self, key_stats_only = False):
        """ Begin tuning """

        print("Begin Guidance")

        self.key_stats_only = key_stats_only

        self._round = 0

        # start by putting original bounds into a list; this list is the object that will control whether algorithm has terminated
        bounds_list = [self.original_bounds]
        centre_down = [False]

        while bounds_list: # gets reset every time, so algo will keep running if there are bounds to operate on

            old_bounds_list = copy.deepcopy(bounds_list)
            bounds_list = list()
            old_centre_down = copy.deepcopy(centre_down)
            centre_down = list()

            for k in range(len(old_bounds_list)): # now run algorithm on every bound

                # get the coordinates that define the bounds
                coords_to_tune = self.get_coords_from_bounds(old_bounds_list[k]) 

                # get all coordinates into a DataFrame - used for getting boundary
                boundaries = pd.DataFrame()
                for coord in coords_to_tune:
                    
                    # combination that goes straight into OLS
                    combo_OLS_dict = {self.hyperparameters[i]:[coord[i]] for i in range(len(self.hyperparameters))}
                    
                    # decide whether to search (criteria: has it been searched before)
                    if tuple(coord) in self.checked_dict:
                        print(f'Already Trained and Tested a combination')
                        combo_OLS_dict['score'] = self.checked_dict[tuple(coord)]['score']
                        
                    else:
                        combo_dict = dict() # combination that gets transformed for searching
                        for i in range(len(self.hyperparameters)):
                    
                            # transform
                            if self.transform[self.hyperparameters[i]] == '10^':
                                combo_dict[self.hyperparameters[i]] = [10**coord[i]]
                    
                            else:
                                combo_dict[self.hyperparameters[i]] = [coord[i]] 

                        # search it
                        self._up_to += 1
                        self._train_and_test_combo(combo_dict)
                        combo_OLS_dict['score'] = [self.val_score] 

                        if self._is_new_best:
                            self._protective_bounds = self._get_protective_bounds(old_bounds_list[k])
                            # TODO: export self._protective_bounds as json

                        # store its metadata into checked_dict
                        self.checked_dict[tuple(coord)] = {'score': self.val_score, 'combo_type': 'Boundary'} #TODO: 可以移除

                    # put this coord into df containing all boundaries (for later sliming depending on centre, and then OLS)
                    tmp_boundary = pd.DataFrame(combo_OLS_dict)
                    boundaries = boundaries.append(tmp_boundary)

                # get the components that make up the centre (as well as new categories); and then unpack them into centres
                centre_components, new_cat = self.get_centre_components(old_bounds_list[k], self.categorical) # 加进去 - 改 for bound bounds with index
                
                centres = self.unpack_centre(centre_components)

                # get the categorical features' values into a list for use in OLS preparation
                categorical_value_list = self.get_categorical(new_cat, boundaries)

                for i in range(len(centres)): # run through each different centre

                    # create a dataframe version of centre (so we could put it into OLS)
                    centre_OLS_df = pd.DataFrame({self.hyperparameters[j]:[centres[i][j]] for j in range(len(centres[i]))})
                    
                    # decide whether to search (criteria: has it been searched before)
                    if tuple(centres[i]) in self.checked_dict:
                        print(f'Already Trained and Tested a combination')
                        combo_OLS_dict['score'] = self.checked_dict[tuple(centres[i])]['score']
                        
                    else:
                        centre_df = dict()
                        for j in range(len(self.hyperparameters)):
                    
                            # transform
                            if self.transform[self.hyperparameters[j]] == '10^':
                                combo_dict[self.hyperparameters[j]] = [10**centres[i][j]]
                    
                            else:
                                combo_dict[self.hyperparameters[j]] = [coord[j]]

                        # search it
                        self._up_to += 1
                        self._train_and_test_combo(centre_df) 
                        actual_centre_score = self.val_score

                        # store its metadata into checked_list
                        self.checked_dict[tuple(centres[i])] = {'score': self.val_score, 'type': 'Centre'} #TODO: 可以移除

                    # copy the boundary dataframes - to turn into the correct training data for OLS (one lm model for each centre)
                    tmp_boundary = copy.deepcopy(boundaries)
                    tmp_boundary_drop = copy.deepcopy(boundaries)
                    
                    n_cat = 0
                    for j in range(len(new_cat)):
                
                        if new_cat[self.hyperparameters[j]] == True:

                            tmp_boundary = tmp_boundary[tmp_boundary[self.hyperparameters[j]] == categorical_value_list[i][n_cat]]
                            tmp_boundary_drop = tmp_boundary_drop[tmp_boundary_drop[self.hyperparameters[j]] == categorical_value_list[i][n_cat]]
                            tmp_boundary_drop = tmp_boundary_drop.drop([self.hyperparameters[j]], axis = 1)
                            centre_OLS_df = centre_OLS_df.drop([self.hyperparameters[j]], axis=1)
                            n_cat += 1

                    tmp_boundary_X = tmp_boundary_drop.drop(['score'], axis = 1)
                    tmp_boundary_y = tmp_boundary_drop['score']

                    OLS = sm.OLS(tmp_boundary_y, tmp_boundary_X).fit()
                    pred_centre_score = OLS.predict(centre_OLS_df)[0]

                    if self._is_new_best:
                        self._protective_bounds = self._get_protective_bounds2(tmp_boundary, new_cat)
                        # TODO: export self._protective_bounds as json


                    if self._rounds == 0: 
                        if actual_centre_score < pred_centre_score - 0.005: # first round + down: mark 'down' and repeat

                            bounds_original_format = self.rebuild_bounds_to_original_format(tmp_boundary, new_cat)
                            new_bounds_list = self.get_new_bounds(bounds_original_format, centres[i], new_cat)
                            bounds_list.extend(new_bounds_list)
                            
                            # record whether centre has gone down
                            tmp_centre_down = [True for j in range(len(new_bounds_list))]
                            centre_down.extend(tmp_centre_down)
                    
                        else: # first round + down OR first round + fits: repeat
                            bounds_original_format = self.rebuild_bounds_to_original_format(tmp_boundary, new_cat)
                            new_bounds_list = self.get_new_bounds(bounds_original_format, centres[i], new_cat)
                            bounds_list.extend(new_bounds_list)
                            
                            # record whether centre has gone down
                            tmp_centre_down = [False for j in range(len(new_bounds_list))]
                            centre_down.extend(tmp_centre_down)
                                
                    
                    else:
                        if actual_centre_score < pred_centre_score - 0.005: # not first round, in protective zone, down: mark 'down' and repeat
                                
                            if self._in_protective_bounds(centres[i]):

                                if old_centre_down[k] == True: # not first round, not in protective zone, double down, down: stop
                                    pass

                                else: # not first round, not in protective zone, not double down, down: mark 'down' and repeat
                                    bounds_original_format = self.rebuild_bounds_to_original_format(tmp_boundary, new_cat)
                                    new_bounds_list = self.get_new_bounds(bounds_original_format, centres[i], new_cat)
                                    bounds_list.extend(new_bounds_list)
                                    
                                    # record whether centre has gone down
                                    tmp_centre_down = [True for j in range(len(new_bounds_list))]
                                    centre_down.extend(tmp_centre_down)
                        
                        elif actual_centre_score > pred_centre_score + 0.005: # not first round, in protective zone, up: repeat
                            bounds_original_format = self.rebuild_bounds_to_original_format(tmp_boundary, new_cat)
                            new_bounds_list = self.get_new_bounds(bounds_original_format, centres[i], new_cat)
                            bounds_list.extend(new_bounds_list)
                            
                            # record whether centre has gone down
                            tmp_centre_down = [False for j in range(len(new_bounds_list))]
                            centre_down.extend(tmp_centre_down)
                        
                        else: # not first round, in protective zone, fits: stop
                            pass
            
            self._round += 1


        # Cruise algorithm
        print("Begin Cruise")

        cruise_bounds = [self._protective_bounds_to_original_bounds()]

        run_through = True

        while cruise_bounds: # gets reset every time, so algo will keep running if there are bounds to operate on
            
            if run_through == True:
                old_max_bounds = cruise_bounds[0]

            old_cruise_bounds = copy.deepcopy(cruise_bounds)
            cruise_bounds = list()

            for k in range(len(old_cruise_bounds)): # now run algorithm on every bound

                # get the coordinates that define the bounds
                coords_to_tune = self.get_coords_from_bounds(old_cruise_bounds[k]) 

                # get all coordinates into a DataFrame - used for getting boundary
                boundaries = pd.DataFrame()
                for coord in coords_to_tune:
                    
                    # combination that goes straight into OLS
                    combo_OLS_dict = {self.hyperparameters[i]:[coord[i]] for i in range(len(self.hyperparameters))}
                    
                    # decide whether to search (criteria: has it been searched before)
                    if tuple(coord) in self.checked_dict:
                        print(f'Already Trained and Tested a combination')
                        combo_OLS_dict['score'] = self.checked_dict[tuple(coord)]['score']
                        
                    else:
                        combo_dict = dict() # combination that gets transformed for searching
                        for i in range(len(self.hyperparameters)):
                    
                            # transform
                            if self.transform[self.hyperparameters[i]] == '10^':
                                combo_dict[self.hyperparameters[i]] = [10**coord[i]]
                    
                            else:
                                combo_dict[self.hyperparameters[i]] = [coord[i]] 

                        # search it
                        self._up_to += 1
                        self._train_and_test_combo(combo_dict)
                        combo_OLS_dict['score'] = [self.val_score] 

                        if self._is_new_best:
                            self._protective_bounds = self._get_protective_bounds(old_cruise_bounds[k])
                            # TODO: export self._protective_bounds as json

                        # store its metadata into checked_dict
                        self.checked_dict[tuple(coord)] = {'score': self.val_score, 'combo_type': 'Boundary'} #TODO: 可以移除

                    # put this coord into df containing all boundaries (for later sliming depending on centre, and then OLS)
                    tmp_boundary = pd.DataFrame(combo_OLS_dict)
                    boundaries = boundaries.append(tmp_boundary)

                # get the components that make up the centre (as well as new categories); and then unpack them into centres
                centre_components, new_cat = self.get_centre_components(old_cruise_bounds[k], self.categorical) # 加进去 - 改 for bound bounds with index
                
                centres = self.unpack_centre(centre_components)

                # get the categorical features' values into a list for use in OLS preparation
                categorical_value_list = self.get_categorical(new_cat, boundaries)

                for i in range(len(centres)): # run through each different centre

                    # create a dataframe version of centre (so we could put it into OLS)
                    centre_OLS_df = pd.DataFrame({self.hyperparameters[j]:[centres[i][j]] for j in range(len(centres[i]))})
                    
                    # decide whether to search (criteria: has it been searched before)
                    if tuple(centres[i]) in self.checked_dict:
                        print(f'Already Trained and Tested a combination')
                        combo_OLS_dict['score'] = self.checked_dict[tuple(centres[i])]['score']
                        
                    else:
                        centre_df = dict()
                        for j in range(len(self.hyperparameters)):
                    
                            # transform
                            if self.transform[self.hyperparameters[j]] == '10^':
                                combo_dict[self.hyperparameters[j]] = [10**centres[i][j]]
                    
                            else:
                                combo_dict[self.hyperparameters[j]] = [coord[j]]

                        # search it
                        self._up_to += 1
                        self._train_and_test_combo(centre_df) 
                        actual_centre_score = self.val_score

                        # store its metadata into checked_list
                        self.checked_dict[tuple(centres[i])] = {'score': self.val_score, 'type': 'Centre'} #TODO: 可以移除

                    # copy the boundary dataframes - to turn into the correct training data for OLS (one lm model for each centre)
                    tmp_boundary = copy.deepcopy(boundaries)
                    tmp_boundary_drop = copy.deepcopy(boundaries)
                    
                    n_cat = 0
                    for j in range(len(new_cat)):
                
                        if new_cat[self.hyperparameters[j]] == True:

                            tmp_boundary = tmp_boundary[tmp_boundary[self.hyperparameters[j]] == categorical_value_list[i][n_cat]]
                            tmp_boundary_drop = tmp_boundary_drop[tmp_boundary_drop[self.hyperparameters[j]] == categorical_value_list[i][n_cat]]
                            tmp_boundary_drop = tmp_boundary_drop.drop([self.hyperparameters[j]], axis = 1)
                            centre_OLS_df = centre_OLS_df.drop([self.hyperparameters[j]], axis=1)
                            n_cat += 1

                    tmp_boundary_X = tmp_boundary_drop.drop(['score'], axis = 1)
                    tmp_boundary_y = tmp_boundary_drop['score']

                    OLS = sm.OLS(tmp_boundary_y, tmp_boundary_X).fit()
                    pred_centre_score = OLS.predict(centre_OLS_df)[0]

                    if self._is_new_best:
                        self._protective_bounds = self._get_protective_bounds2(tmp_boundary, new_cat)
                        # TODO: export self._protective_bounds as json

                    if run_through == True:
                        # run straight through - one more round
                        bounds_original_format = self.rebuild_bounds_to_original_format(tmp_boundary, new_cat)
                        new_bounds_list = self.get_new_bounds(bounds_original_format, centres[i], new_cat)
                        cruise_bounds.extend(new_bounds_list)

            if run_through == False:
                max_bounds = self._protective_bounds_to_original_bounds()
                if max_bounds != old_max_bounds:
                    cruise_bounds.append(max_bounds)

            run_through = not run_through



    def _train_and_test_combo(self, combo):
        """ Helper to train and test each combination as part of tune() """
        
        combo = tuple(combo)
        
        params = {self.hyperparameters[i]:self.parameter_choices[self.hyperparameters[i]][combo[i]] for i in range(len(self.hyperparameters))}
        
        
        if self._tune_features == True:
            del params['features']
            tmp_train_x = self.train_x[list(self.feature_combo_n_index_map[combo[-1]])] 
            tmp_val_x = self.val_x[list(self.feature_combo_n_index_map[combo[-1]])]
            tmp_test_x = self.test_x[list(self.feature_combo_n_index_map[combo[-1]])]

            # add non tuneable parameters
            for nthp in self.non_tuneable_parameter_choices:
                params[nthp] = self.non_tuneable_parameter_choices[nthp]

            # initialise object
            clf = self.model(**params)

            params['features'] = [list(self.feature_combo_n_index_map[combo[-1]])] 
            params['feature combo ningxiang score'] = self.feature_n_ningxiang_score_dict[self.feature_combo_n_index_map[combo[-1]]]

        else:
            tmp_train_x = self.train_x
            tmp_val_x = self.val_x
            tmp_test_x = self.test_x

            # add non tuneable parameters
            for nthp in self.non_tuneable_parameter_choices:
                params[nthp] = self.non_tuneable_parameter_choices[nthp]

            # initialise object
            clf = self.model(**params)

        # get time and fit
        start = time.time()
        clf.fit(tmp_train_x, self.train_y)
        end = time.time()

        # get predicted labels/values for three datasets
        train_pred = clf.predict(tmp_train_x)
        val_pred = clf.predict(tmp_val_x)
        test_pred = clf.predict(tmp_test_x)

        # get scores and time used
        time_used = end-start

        # build output dictionary and save result
        df_building_dict = params
        for nthp in self.non_tuneable_parameter_choices:
            del params[nthp] 


        if self.clf_type == 'Regression':

            train_score = val_score = test_score = train_rmse = val_rmse = test_rmse = train_mape = val_mape = test_mape = 0

            try:
                train_score = r2_score(self.train_y, train_pred)
            except:
                pass
            try:
                val_score = r2_score(self.val_y, val_pred)
            except:
                pass
            try:
                test_score = r2_score(self.test_y, test_pred)
            except:
                pass
            
            try:
                train_rmse = np.sqrt(mean_squared_error(self.train_y, train_pred))
            except:
                pass
            try:
                val_rmse = np.sqrt(mean_squared_error(self.val_y, val_pred))
            except:
                pass
            try:
                test_rmse = np.sqrt(mean_squared_error(self.test_y, test_pred))
            except:
                pass

            if self.key_stats_only == False:
                try:
                    train_mape = mean_absolute_percentage_error(self.train_y, train_pred)
                except:
                    pass
                try:
                    val_mape = mean_absolute_percentage_error(self.val_y, val_pred)
                except:
                    pass
                try:
                    test_mape = mean_absolute_percentage_error(self.test_y, test_pred)
                except:
                    pass
            
            df_building_dict['Train r2'] = [np.round(train_score, 6)]
            df_building_dict['Val r2'] = [np.round(val_score, 6)]
            df_building_dict['Test r2'] = [np.round(test_score, 6)]
            df_building_dict['Train RMSE'] = [np.round(train_rmse, 6)]
            df_building_dict['Val RMSE'] = [np.round(val_rmse, 6)]
            df_building_dict['Test RMSE'] = [np.round(test_rmse, 6)]
            
            if self.key_stats_only == False:
                df_building_dict['Train MAPE'] = [np.round(train_mape, 6)]
                df_building_dict['Val MAPE'] = [np.round(val_mape, 6)]
                df_building_dict['Test MAPE'] = [np.round(test_mape, 6)]

        
        elif self.clf_type == 'Classification':

            train_score = val_score = test_score = train_bal_accu = val_bal_accu = test_bal_accu = train_f1 = val_f1 = test_f1 = \
                train_precision = val_precision = test_precision = train_recall = val_recall = test_recall = 0

            try:    
                train_score = accuracy_score(self.train_y, train_pred)
            except:
                pass
            try:
                val_score = accuracy_score(self.val_y, val_pred)
            except:
                pass
            try:
                test_score = accuracy_score(self.test_y, test_pred)
            except:
                pass

            try:
                train_bal_accu = balanced_accuracy_score(self.train_y, train_pred)
            except:
                pass
            try:
                val_bal_accu = balanced_accuracy_score(self.val_y, val_pred)
            except:
                pass
            try:
                test_bal_accu = balanced_accuracy_score(self.test_y, test_pred)
            except:
                pass
            
            try:
                train_f1 = f1_score(self.train_y, train_pred, average='weighted')
            except:
                pass
            try:
                val_f1 = f1_score(self.val_y, val_pred, average='weighted')
            except:
                pass
            try:
                test_f1 = f1_score(self.test_y, test_pred, average='weighted')
            except:
                pass
            
            try:
                train_precision = precision_score(self.train_y, train_pred, average='weighted')
            except:
                pass
            try:
                val_precision = precision_score(self.val_y, val_pred, average='weighted')
            except:
                pass
            try:
                test_precision = precision_score(self.test_y, test_pred, average='weighted')
            except:
                pass

            try:
                train_recall = recall_score(self.train_y, train_pred, average='weighted')
            except:
                pass
            try:
                val_recall = recall_score(self.val_y, val_pred, average='weighted')
            except:
                pass
            try:
                test_recall = recall_score(self.test_y, test_pred, average='weighted')
            except:
                pass

            df_building_dict['Train accu'] = [np.round(train_score, 6)]
            df_building_dict['Val accu'] = [np.round(val_score, 6)]
            df_building_dict['Test accu'] = [np.round(test_score, 6)]
            df_building_dict['Train balanced_accuracy'] = [np.round(train_bal_accu, 6)]
            df_building_dict['Val balanced_accuracy'] = [np.round(val_bal_accu, 6)]
            df_building_dict['Test balanced_accuracy'] = [np.round(test_bal_accu, 6)]
            df_building_dict['Train f1'] = [np.round(train_f1, 6)]
            df_building_dict['Val f1'] = [np.round(val_f1, 6)]
            df_building_dict['Test f1'] = [np.round(test_f1, 6)]
            df_building_dict['Train precision'] = [np.round(train_precision, 6)]
            df_building_dict['Val precision'] = [np.round(val_precision, 6)]
            df_building_dict['Test precision'] = [np.round(test_precision, 6)]
            df_building_dict['Train recall'] = [np.round(train_recall, 6)]
            df_building_dict['Val recall'] = [np.round(val_recall, 6)]
            df_building_dict['Test recall'] = [np.round(test_recall, 6)]


        df_building_dict['Time'] = [np.round(time_used, 2)]


        tmp = pd.DataFrame(df_building_dict)

        self.tuning_result = self.tuning_result.append(tmp)
        self._save_tuning_result()

        self._is_new_best = 0

        # update best score stats
        if val_score > self.best_score: 
            self.best_score = val_score
            self.best_clf = clf
            self.best_combo = combo

            self._is_new_best = 1

        # update internal governing DataFrames
        self.checked[combo] = 1
        self.result[combo] = val_score

        # add a new self variable compared to previous JiaXing classes
        self.val_score = val_score

        print(f'''Trained and Tested combination {self._up_to}: {combo}, taking {time_used} seconds to get val score of {val_score}
        Current best combo: {self.best_combo} with val score {self.best_score}''')



    def _save_tuning_result(self):
        """ Helper to export tuning result csv """

        tuning_result_saving_address_strip = self.tuning_result_saving_address.split('.csv')[0]

        self.tuning_result.to_csv(f'{tuning_result_saving_address_strip}.csv', index=False)


    
    def view_best_combo_and_score(self):
        """ View best combination and its validation score """
        
        print(f'(Current) Best combo: {self.best_combo} with val score {self.best_score}')

    

    def read_in_tuning_result_df(self, address): 
        """ Read in tuning result csv and read data into checked and result arrays """

        if self.parameter_choices is None:
            print("Missing parameter_choices to build parameter_value_map_index, please run set_hyperparameters() first")

        if self.clf_type is None:
            print('Missing clf_type. Please run .read_in_model() first.')

        self.tuning_result = pd.read_csv(address)
        print(f"Successfully read in tuning result of {len(self.tuning_result)} rows")
        
        self._up_to = 0

        # read DataFrame data into internal governing DataFrames of GuangAn
        for row in self.tuning_result.iterrows():
            
            self._up_to += 1

            combo = tuple([row[1][hyperparam] for hyperparam in self.hyperparameters])
            
            if self.clf_type == 'Regression':
                self.checked[combo] = {'score': row[1]['Val r2']}
            elif self.clf_type == 'Classification':
                self.checked[combo] = {'score': row[1]['Val r2']}
        
            # update best score stats
            if self.checked[combo]['score'] > self.best_score: 
                self.best_score = self.checked[combo]['score']
                self.best_clf = None
                print(f"As new Best Combo {combo} is read in, best_clf is set to None")
                self.best_combo = combo
    


    def set_tuning_result_saving_address(self, address):
        """ Read in where to save tuning object """

        self.tuning_result_saving_address = address
        print('Successfully set tuning output address')


    
    def _set_object_saving_address(self, address):
        """ Read in where to save the GuangAn object """

        self.object_saving_address = address
        print('Successfully set object output address')



    def export_guangan(self, address):
        """ Export GuangAn object """

        self._set_object_saving_address(address)

        # copy object and set big objects to None
        object_save = copy.deepcopy(self)
        
        object_save.train_x = None
        object_save.train_y = None
        object_save.val_x = None
        object_save.val_y = None
        object_save.test_x = None
        object_save.test_y = None
        object_save._up_to = 0

        # Export
        object_saving_address_strip = self.object_saving_address.split('.pickle')[0]
        with open(f'{object_saving_address_strip}.pickle', 'wb') as f:
            pickle.dump(object_save, f)

        print(f'Successfully exported GuangAn object as {self.object_saving_address}')

In [24]:
guangan = GuangAn()

GuangAn Initialised


In [1]:
parameter_ranges = {
    'A': (0, 10),
    'B': (-4, 2),
    'C': {0, 1, 2, 3, 4},
    'D': {'a', 'b'}
}
# parameter_ranges = {
#     'C': {0, 1, 2, 3, 4},
#     'D': {'a', 'b'},
#     'A': (0, 10),
#     'B': (-4, 2),
# }
# parameter_ranges = {
#     'D': {'a', 'b'},
#     'A': (0, 10),
#     'B': (-4, 2),
#     'C': {0, 1, 2, 3, 4},
# }

categorical = ['D'] # 自动化？
transform = {'B': '10^'}

In [2]:
guangan.set_hyperparameters(parameter_ranges)

NameError: name 'guangan' is not defined

In [49]:
guangan.read_in_categorical(categorical)

Updated categorical dictionary: {'D': True, 'A': False, 'B': False, 'C': False}
Updated original bounds dict: [({'values': ('a', 'b')}, 'D'), ((0, 10), 'A'), ((-4, 2), 'B'), ({0, 1, 2, 3, 4}, 'C')]


In [50]:
guangan.read_in_transform(transform)

Updated transform dictionary: {'D': False, 'A': False, 'B': '10^', 'C': False}


In [51]:
guangan.parameter_ranges

{'D': {'values': ('a', 'b')}, 'A': (0, 10), 'B': (-4, 2), 'C': {0, 1, 2, 3, 4}}

In [10]:
a = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})